- Sleep stage W (覚醒状態)
- Sleep stage 1
- Sleep stage 2
- Sleep stage 3
- Sleep stage 4
- Sleep stage R (レム睡眠)
- Sleep stage ? (不明)
- Movement time

In [1]:
import datetime
from typing import Dict, List
import pandas as pd
import numpy as np
import warnings
from tqdm.auto import tqdm

from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score

import mne

c:\Users\MK\Anaconda3\envs\kaggle\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sample_submission_df = pd.read_csv("sample_submission.csv", parse_dates=[1])
sample_submission_df.head()

,id,meas_time,condition
0,53c1555,1989-11-20 23:19:30,Sleep stage W
1,53c1555,1989-11-20 23:20:00,Sleep stage W
2,53c1555,1989-11-20 23:20:30,Sleep stage W
3,53c1555,1989-11-20 23:21:00,Sleep stage W
4,53c1555,1989-11-20 23:21:30,Sleep stage W


- meas_timeは睡眠段階の予測時間
- 30秒を1epochとして睡眠段階を判断しているため、30秒毎に行がある
- 提供されているデータは波形データとアノテーションデータが別々になっているため、この形式に処理する必要がある

In [3]:
train_record_df = pd.read_csv("train_records.csv")
test_record_df = pd.read_csv("test_records.csv")
train_record_df.head()

,id,subject_id,night,age,sex,lights_off,psg,hypnogram
0,3c1c5cf,07c46da,1,90,male,23:28:00,3c1c5cf-PSG.edf,3c1c5cf-Hypnogram.edf
1,8fbd71b,07c46da,2,90,male,01:29:00,8fbd71b-PSG.edf,8fbd71b-Hypnogram.edf
2,9d5e9ec,21969ff,1,51,female,23:10:00,9d5e9ec-PSG.edf,9d5e9ec-Hypnogram.edf
3,e0df8c0,21969ff,2,51,female,23:15:00,e0df8c0-PSG.edf,e0df8c0-Hypnogram.edf
4,3e404fc,22b58e8,1,51,female,22:38:00,3e404fc-PSG.edf,3e404fc-Hypnogram.edf


In [4]:
test_record_df.head()

,id,subject_id,night,age,sex,lights_off,psg
0,53c1555,17ca2cd,1,91,female,00:15:00,53c1555-PSG.edf
1,29ef1d5,17ca2cd,2,91,female,23:39:00,29ef1d5-PSG.edf
2,c90b6e7,2c77159,1,56,female,23:55:00,c90b6e7-PSG.edf
3,a61e635,2c77159,2,56,female,00:13:00,a61e635-PSG.edf
4,2cb6860,40dc0bc,1,52,male,23:03:00,2cb6860-PSG.edf


In [5]:
# trainとtestのsubject_idが被っているものがないか確認
len(set(train_record_df["subject_id"].unique()) & set(test_record_df["subject_id"].unique()))

0

In [6]:
print("訓練被験者数", len(train_record_df["subject_id"].unique()))
print("テスト被験者数", len(test_record_df["subject_id"].unique()))

訓練被験者数 55
テスト被験者数 23


## 被験者データの表示

In [7]:
# パスを設定
EDF_DIR = "edf_data"
train_record_df["hypnogram"] = train_record_df["hypnogram"].map(lambda x: f"{EDF_DIR}/{x}")
train_record_df["psg"] = train_record_df["psg"].map(lambda x: f"{EDF_DIR}/{x}")
test_record_df["psg"] = test_record_df["psg"].map(lambda x: f"{EDF_DIR}/{x}")

In [8]:
train_record_df.head()

,id,subject_id,night,age,sex,lights_off,psg,hypnogram
0,3c1c5cf,07c46da,1,90,male,23:28:00,edf_data/3c1c5cf-PSG.edf,edf_data/3c1c5cf-Hypnogram.edf
1,8fbd71b,07c46da,2,90,male,01:29:00,edf_data/8fbd71b-PSG.edf,edf_data/8fbd71b-Hypnogram.edf
2,9d5e9ec,21969ff,1,51,female,23:10:00,edf_data/9d5e9ec-PSG.edf,edf_data/9d5e9ec-Hypnogram.edf
3,e0df8c0,21969ff,2,51,female,23:15:00,edf_data/e0df8c0-PSG.edf,edf_data/e0df8c0-Hypnogram.edf
4,3e404fc,22b58e8,1,51,female,22:38:00,edf_data/3e404fc-PSG.edf,edf_data/3e404fc-Hypnogram.edf


In [9]:
row = train_record_df.iloc[0]
row

id                                   3c1c5cf
subject_id                           07c46da
night                                      1
age                                       90
sex                                     male
lights_off                          23:28:00
psg                 edf_data/3c1c5cf-PSG.edf
hypnogram     edf_data/3c1c5cf-Hypnogram.edf
Name: 0, dtype: object

In [10]:
# edfファイルの読み込み
psg_edf = mne.io.read_raw_edf(row["psg"], preload=False)
# 読み込んだデータは、mne.io.edf.edf.RawEDFクラスのインスタンスになります
type(psg_edf)

Extracting EDF parameters from c:\Users\MK\workspace\kaggle\nishika-sleep\edf_data\3c1c5cf-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


mne.io.edf.edf.RawEDF

In [11]:
# infoでメタ情報を表示できます
psg_edf.info

<Info | 7 non-empty values
 bads: []
 ch_names: EEG Fpz-Cz, EEG Pz-Oz, EOG horizontal, Resp oro-nasal, EMG ...
 chs: 7 EEG
 custom_ref_applied: False
 highpass: 0.5 Hz
 lowpass: 100.0 Hz
 meas_date: 1989-11-13 16:35:00 UTC
 nchan: 7
 projs: []
 sfreq: 100.0 Hz
>

In [12]:
# センサーチャンネルの名前を確認
psg_edf.ch_names

['EEG Fpz-Cz',
 'EEG Pz-Oz',
 'EOG horizontal',
 'Resp oro-nasal',
 'EMG submental',
 'Temp rectal',
 'Event marker']

- EEG： 脳波。すべての予測段階の判定に必要であり、意識水準と対応して変化する。
  - Fpz-Cz、Pz-Ozは頭に取り付けるセンサー箇所の箇所の電位差を表す
- EOG： 眼電図。 眼球運動を測定。
  - horizontalは水平方向に取り付けた電位差を表す
- EMG： オトガイ(顎下)筋電図。
- Resp oro-nasal：口鼻呼吸 (oroが口、nasalが鼻、respはrespirationの略で呼吸を表す)
- Temp rectal：直腸温 (原著に記載がないため、実際に直腸の温度を計測したかどうかは不明)
- Event marker：各時刻で起きたイベントの時刻

In [13]:
psg_df = psg_edf.to_data_frame()
psg_df.head()

,time,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker
0,0.00,84.851770,-11.197558,-219.482051,8.800000e+07,3.234000,1.427420e+07,9.490000e+08
1,0.01,87.957998,13.197558,-215.531868,8.790847e+07,3.233153,1.428125e+07,9.499478e+08
2,0.02,96.862515,-3.908181,-209.112821,8.781710e+07,3.232296,1.428833e+07,9.509124e+08
3,0.03,106.181197,-2.644689,-204.175092,8.772580e+07,3.231429,1.429543e+07,9.518927e+08
4,0.04,102.557265,-25.095971,-198.249817,8.763449e+07,3.230552,1.430254e+07,9.528878e+08


- EEGとEOGは100Hzなので、0.00秒から始まっています。
- 従って、データの合計時間は(7890000/100)/3600=約22時間です。
- 計測時間の開始はmeas_dateで取得できます
- これを設定すると16時から翌14時過ぎまで計測していることが分かります

In [14]:
meas_start = psg_edf.info["meas_date"]
meas_start = meas_start.replace(tzinfo=None)
# 100Hz
psg_df["meas_time"] = pd.date_range(start=meas_start, periods=len(psg_df), freq=pd.Timedelta(1 / 100, unit="s"))
psg_df.head()

,time,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker,meas_time
0,0.00,84.851770,-11.197558,-219.482051,8.800000e+07,3.234000,1.427420e+07,9.490000e+08,1989-11-13 16:35:00.000
1,0.01,87.957998,13.197558,-215.531868,8.790847e+07,3.233153,1.428125e+07,9.499478e+08,1989-11-13 16:35:00.010
2,0.02,96.862515,-3.908181,-209.112821,8.781710e+07,3.232296,1.428833e+07,9.509124e+08,1989-11-13 16:35:00.020
3,0.03,106.181197,-2.644689,-204.175092,8.772580e+07,3.231429,1.429543e+07,9.518927e+08,1989-11-13 16:35:00.030
4,0.04,102.557265,-25.095971,-198.249817,8.763449e+07,3.230552,1.430254e+07,9.528878e+08,1989-11-13 16:35:00.040


## Hypnogramデータ(ラベル)の読み込み

In [15]:
# Hypnogramはread_annotationsで読み込むことが可能です
annot = mne.read_annotations(row["hypnogram"])
annot_df = annot.to_data_frame()
annot_df.head()

,onset,duration,description
0,1970-01-01 00:00:00,16290.0,Sleep stage W
1,1970-01-01 04:31:30,30.0,Sleep stage 1
2,1970-01-01 04:32:00,90.0,Sleep stage 2
3,1970-01-01 04:33:30,1890.0,Sleep stage W
4,1970-01-01 05:05:00,30.0,Sleep stage 1


- onset：経過時間（年と日付は設定が必要なので正しくありません。この例では時間の経過のみが正しいものになります）
- duration：アノテーションの間隔時間
- description：睡眠段階のラベル

最初と最後にかなり長いSleep Stage Wがあることが分かります

In [16]:
annot_df["description"].value_counts()

Sleep stage 1    47
Sleep stage 2    39
Sleep stage W    28
Sleep stage 3    12
Sleep stage R     8
Sleep stage ?     3
Name: description, dtype: int64

## 波形データとアノテーションデータの紐づけ

- 波形データとアノテーションデータは別々のデータ構造を持っているため、波形データにアノテーションを紐付ける作業が必要
- 睡眠段階は30秒を1epochとして判定されるため、波形データを30秒毎に区切って1epochとして、onsetとdurationから睡眠段階を紐付ける
- mneには、この操作を行うメソッドがある
- また、前後にかなり長いSleep Stage Wがある
- データ数を減らすため、適当に5時間で解析時間の切り捨てを行う
- 今回は処理を簡単にするために、チャンネルにEEG Fpz-Czのみを利用(read_raw_edfのincludeメソッドにEEGのみを指定)

In [17]:
# ラベル名をIDに置き換える
# Sleep stage 3とSleep stage 4を同じIDとして、AASMによる分類に変更する
RANDK_LABEL2ID = {
    'Movement time': -1,
    'Sleep stage ?': -1,
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3': 3,
    'Sleep stage 4': 3,
    'Sleep stage R': 4
}

# AASMによる分類
LABEL2ID = {
    'Movement time': -1,
    'Sleep stage ?': -1,
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3/4': 3,
    'Sleep stage R': 4
}
ID2LABEL = {v:k for k, v in LABEL2ID.items()}

In [18]:
# 再度データを読み込みます
# 簡単のためにEEG Fpz-Czのみ利用します
psg_edf = mne.io.read_raw_edf(row["psg"], include=["EEG Fpz-Cz"], verbose=False)
annot = mne.read_annotations(row["hypnogram"])

# 1時間(3600秒)*5
truncate_start_point = 3600 * 5
truncate_end_point = (len(psg_edf)/100) - (3600 *5)
# 切り捨て
annot.crop(truncate_start_point, truncate_end_point, verbose=False)

# annotationデータを紐づけます
psg_edf.set_annotations(annot, verbose=False, emit_warning=False)

# 30秒毎に分割された睡眠段階
events, _ = mne.events_from_annotations(psg_edf, event_id=RANDK_LABEL2ID, chunk_duration=30., verbose=False)

# 30秒毎に1epochとする
tmax = 30. - 1. / psg_edf.info['sfreq']
epoch = mne.Epochs(raw=psg_edf, events=events, event_id=LABEL2ID, tmin=0, tmax=tmax, baseline=None, verbose=False, on_missing='ignore')

# subjectとnightの設定
epoch.info["temp"] = {
    "id":row["id"],
    "subject_id":row["subject_id"],
    "night":row["night"],
    "truncate_start_point":truncate_start_point,
    "truncate_end_point":truncate_end_point
}

In [19]:
print(type(epoch))
epoch

<class 'mne.epochs.Epochs'>


Number of events,1430
Events,Movement time: 20Sleep stage 1: 122Sleep stage 2: 422Sleep stage 3/4: 27Sleep stage ?: 20Sleep stage R: 35Sleep stage W: 804
Time range,0.000 – 29.990 sec
Baseline,off


In [20]:
# RawEDFクラスと同様にデータフレーム化できます
epoch_df = epoch.to_data_frame(verbose=False)
epoch_df

,time,condition,epoch,EEG Fpz-Cz
0,0.00,Sleep stage W,0,-80.295971
1,0.01,Sleep stage W,0,-32.252991
2,0.02,Sleep stage W,0,-88.165079
3,0.03,Sleep stage W,0,15.893529
4,0.04,Sleep stage W,0,23.555556
...,...,...,...,...
4289995,29.95,Sleep stage 1,1429,7.092552
4289996,29.96,Sleep stage 1,1429,0.155311
4289997,29.97,Sleep stage 1,1429,6.574847
4289998,29.98,Sleep stage 1,1429,-6.678388


In [21]:
events

array([[1800000,       0,       0],
       [1803000,       0,       0],
       [1806000,       0,       0],
       ...,
       [6081000,       0,       0],
       [6084000,       0,       1],
       [6087000,       0,       1]])

In [22]:
# epoch数と一致することが分かります
events.shape

(1430, 3)

## 計測時間の設定

In [23]:
# 測定日を切り捨てが発生したポイントまでスライド
new_meas_date = epoch.info["meas_date"].replace(tzinfo=None) + datetime.timedelta(seconds=epoch.info["temp"]["truncate_start_point"])
# 連続した時間を作成
epoch_df["meas_time"] = pd.date_range(start=new_meas_date, periods=len(epoch_df), freq=pd.Timedelta(1 / 100, unit="s"))
epoch_df.head()

,time,condition,epoch,EEG Fpz-Cz,meas_time
0,0.00,Sleep stage W,0,-80.295971,1989-11-13 21:35:00.000
1,0.01,Sleep stage W,0,-32.252991,1989-11-13 21:35:00.010
2,0.02,Sleep stage W,0,-88.165079,1989-11-13 21:35:00.020
3,0.03,Sleep stage W,0,15.893529,1989-11-13 21:35:00.030
4,0.04,Sleep stage W,0,23.555556,1989-11-13 21:35:00.040


In [24]:
def epoch_to_df(epoch:mne.epochs.Epochs) -> pd.DataFrame:
    truncate_start_point = epoch.info["temp"]["truncate_start_point"]
    df = epoch.to_data_frame(verbose=False)
    new_meas_date = epoch.info["meas_date"].replace(tzinfo=None) + datetime.timedelta(seconds=truncate_start_point)
    df["meas_time"] = pd.date_range(start=new_meas_date, periods=len(df), freq=pd.Timedelta(1 / 100, unit="s"))
    return df

epoch_df = epoch_to_df(epoch)
epoch_df.head()

,time,condition,epoch,EEG Fpz-Cz,meas_time
0,0.00,Sleep stage W,0,-80.295971,1989-11-13 21:35:00.000
1,0.01,Sleep stage W,0,-32.252991,1989-11-13 21:35:00.010
2,0.02,Sleep stage W,0,-88.165079,1989-11-13 21:35:00.020
3,0.03,Sleep stage W,0,15.893529,1989-11-13 21:35:00.030
4,0.04,Sleep stage W,0,23.555556,1989-11-13 21:35:00.040


In [25]:
# csvに書き出し
epoch_df.to_csv("epoch.csv", index=False)

In [26]:
label_df = epoch_df.loc[epoch_df.groupby("epoch")["time"].idxmin()][["meas_time"]].reset_index(drop=True)
label_df["condition"] = "Sleep stage W"
label_df["id"] = epoch.info["temp"]["id"]
label_df

,meas_time,condition,id
0,1989-11-13 21:35:00,Sleep stage W,3c1c5cf
1,1989-11-13 21:35:30,Sleep stage W,3c1c5cf
2,1989-11-13 21:36:00,Sleep stage W,3c1c5cf
3,1989-11-13 21:36:30,Sleep stage W,3c1c5cf
4,1989-11-13 21:37:00,Sleep stage W,3c1c5cf
...,...,...,...
1425,1989-11-14 09:27:30,Sleep stage W,3c1c5cf
1426,1989-11-14 09:28:00,Sleep stage W,3c1c5cf
1427,1989-11-14 09:28:30,Sleep stage W,3c1c5cf
1428,1989-11-14 09:29:00,Sleep stage W,3c1c5cf


In [27]:
def epoch_to_sub_df(epoch_df:pd.DataFrame, id, is_train:bool) -> pd.DataFrame:
    cols = ["id", "meas_time"]
    if is_train:
        # 訓練セットの場合はラベルを追加
        cols.append("condition")
    label_df = epoch_df.loc[epoch_df.groupby("epoch")["time"].idxmin()].reset_index(drop=True)
    label_df["id"] = id
    return label_df[cols]

label_df = epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], True)
label_df

,id,meas_time,condition
0,3c1c5cf,1989-11-13 21:35:00,Sleep stage W
1,3c1c5cf,1989-11-13 21:35:30,Sleep stage W
2,3c1c5cf,1989-11-13 21:36:00,Sleep stage W
3,3c1c5cf,1989-11-13 21:36:30,Sleep stage W
4,3c1c5cf,1989-11-13 21:37:00,Sleep stage W
...,...,...,...
1425,3c1c5cf,1989-11-14 09:27:30,Sleep stage W
1426,3c1c5cf,1989-11-14 09:28:00,Sleep stage W
1427,3c1c5cf,1989-11-14 09:28:30,Sleep stage W
1428,3c1c5cf,1989-11-14 09:29:00,Sleep stage 1


- テストデータにはアノテーションファイルがないので`events`の行列を、先程の例で作成することはできません。
- 評価の開始と終了時刻はsample submissionにあるので、前後の切り捨て時刻を計算して求めます。
- アノテーションのラベルIDは`0`としてダミーで生成します。

In [28]:
test_row = test_record_df.iloc[0]
psg_edf = mne.io.read_raw_edf(test_row["psg"], include=["EEG Fpz-Cz"], verbose=False)

# PSGの開始をedfのmeas_dateから取得します
start_psg_date = psg_edf.info["meas_date"]
# 計算のためにtimezoneを消去します
start_psg_date = start_psg_date.replace(tzinfo=None)

# sample submissionから評価される時間レンジを取得します
test_start_time = sample_submission_df[sample_submission_df["id"]==test_row["id"]]["meas_time"].min()
test_end_time = sample_submission_df[sample_submission_df["id"]==test_row["id"]]["meas_time"].max()
# psgの計測開始、評価の対象の開始、評価の対象の終了
print(f"psg start: {start_psg_date},  test start: {test_start_time}, test end: {test_end_time}")

# psgの計測時間から評価の対象の開始と終了を経過時間(秒)になおす
truncate_start_point = int((test_start_time - start_psg_date).total_seconds())
truncate_end_point = int((test_end_time - start_psg_date).total_seconds())+30
print(f"event start sencond: {truncate_start_point}, event end second: {truncate_end_point} ")

# 30秒毎にデータ点を生成
event_range = list(range(truncate_start_point, truncate_end_point, 30))
events = np.zeros((len(event_range), 3), dtype=int)
events[:, 0] = event_range

# 秒を10m秒に変換する
events = events * 100

psg start: 1989-11-20 15:16:00,  test start: 1989-11-20 23:19:30, test end: 1989-11-21 08:10:30
event start sencond: 29010, event end second: 60900 


In [29]:
events

array([[2901000,       0,       0],
       [2904000,       0,       0],
       [2907000,       0,       0],
       ...,
       [6081000,       0,       0],
       [6084000,       0,       0],
       [6087000,       0,       0]])

In [30]:
tmax = 30. - 1. / psg_edf.info['sfreq']
epoch = mne.Epochs(raw=psg_edf, events=events, event_id={'Sleep stage W': 0}, tmin=0, tmax=tmax, baseline=None, verbose=False)

In [31]:
epoch.to_data_frame()

Loading data for 1063 events and 3000 original time points ...
0 bad epochs dropped


,time,condition,epoch,EEG Fpz-Cz
0,0.00,Sleep stage W,0,-10.738462
1,0.01,Sleep stage W,0,-13.220513
2,0.02,Sleep stage W,0,-31.481319
3,0.03,Sleep stage W,0,-10.915751
4,0.04,Sleep stage W,0,-39.902564
...,...,...,...,...
3188995,29.95,Sleep stage W,1062,-37.065934
3188996,29.96,Sleep stage W,1062,-34.761172
3188997,29.97,Sleep stage W,1062,-37.331868
3188998,29.98,Sleep stage W,1062,-22.882784


In [32]:
# trainとtestで以上の処理を行えるように関数化
def read_and_set_annoation(record_df:pd.DataFrame, include=None, is_test=False) -> List[mne.epochs.Epochs]:
    whole_epoch_data = []

    for row_id, row in tqdm(record_df.iterrows(), total=len(record_df)):
        # PSGファイルとHypnogram(アノテーションファイルを読み込む)
        psg_edf = mne.io.read_raw_edf(row["psg"], include=include, verbose=False)

        if not is_test:
            # 訓練データの場合
            annot = mne.read_annotations(row["hypnogram"])

            # 切り捨て
            truncate_start_point = 3600 * 5
            truncate_end_point = (len(psg_edf)/100) - (3600 *5)
            annot.crop(truncate_start_point, truncate_end_point, verbose=False)

            # アノテーションデータの切り捨て
            psg_edf.set_annotations(annot, emit_warning=False)
            events, _ = mne.events_from_annotations(psg_edf, event_id=RANDK_LABEL2ID, chunk_duration=30., verbose=False)

            event_id = LABEL2ID
        else:
            # テストデータの場合
            start_psg_date = psg_edf.info["meas_date"]
            start_psg_date = start_psg_date.replace(tzinfo=None)

            test_start_time = sample_submission_df[sample_submission_df["id"]==row["id"]]["meas_time"].min()
            test_end_time = sample_submission_df[sample_submission_df["id"]==row["id"]]["meas_time"].max()

            truncate_start_point = int((test_start_time - start_psg_date).total_seconds())
            truncate_end_point = int((test_end_time- start_psg_date).total_seconds())+30

            event_range = list(range(truncate_start_point, truncate_end_point, 30))
            events = np.zeros((len(event_range), 3), dtype=int)
            events[:, 0] = event_range
            events = events * 100

            event_id = {'Sleep stage W': 0}

        # 30秒毎に1epochとする
        tmax = 30. - 1. / psg_edf.info['sfreq']
        epoch = mne.Epochs(raw=psg_edf, events=events, event_id=event_id, tmin=0, tmax=tmax, baseline=None, verbose=False, on_missing='ignore')

        # 途中でデータが落ちてないかチェック
        assert len(epoch.events) * 30 == truncate_end_point - truncate_start_point

        # メタデータを追加
        epoch.info["temp"] = {
            "id":row["id"],
            "subject_id":row["subject_id"],
            "night":row["night"],
            "age":row["age"],
            "sex":row["sex"],
            "truncate_start_point":truncate_start_point
        }

        whole_epoch_data.append(epoch)

    return whole_epoch_data

In [33]:
# 処理を簡単にするためにEEG Fpz-Czのみ読み込みます
# またtrainをすべて処理するには少し時間がかかるため、ここでは半分ほどの50を利用することにします
train_record_subset_df = train_record_df.sample(n=50).reset_index(drop=True)

train_subset_epoch = read_and_set_annoation(train_record_subset_df, include=["EEG Fpz-Cz"], is_test=False)
test_whole_epoch = read_and_set_annoation(test_record_df, include=["EEG Fpz-Cz"], is_test=True)

100%|██████████| 45/45 [00:01<00:00, 27.16it/s]


## epochとlabelの関係の可視化

In [34]:
import plotly.graph_objects as go

In [35]:
sample_events = train_subset_epoch[0].events[:, 2]
sample_epoch_df = train_subset_epoch[0].to_data_frame(verbose=False)

In [36]:
go.Figure(
    data=[
        go.Scatter(x=sample_epoch_df["epoch"].unique(), y=list(map(lambda x: ID2LABEL[x], sample_events)))
    ],
    layout=go.Layout(
        yaxis=dict(title="sleep stage"),
        xaxis=dict(title="epoch"),
    )
)

In [37]:
epoch_grouped_df = sample_epoch_df.groupby("epoch").agg({"EEG Fpz-Cz":"mean"}).reset_index()
epoch_grouped_df

,epoch,EEG Fpz-Cz
0,0,-0.045046
1,1,0.069662
2,2,-0.561323
3,3,0.162277
4,4,0.171200
...,...,...
1599,1599,0.291969
1600,1600,0.267015
1601,1601,-0.725200
1602,1602,0.958277


In [38]:
go.Figure(
    data=[
        go.Scatter(x=epoch_grouped_df["epoch"], y=epoch_grouped_df["EEG Fpz-Cz"]),
    ],
    layout=go.Layout(
        yaxis=dict(title="EEG Fpz-Cz"),
        xaxis=dict(title="epoch"),
    )
)

## 特徴量エンジニアリング

In [39]:
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data.
    """
    # specific frequency bands
    FREQ_BANDS = {"delta": [0.5, 4.5],
                  "theta": [4.5, 8.5],
                  "alpha": [8.5, 11.5],
                  "sigma": [11.5, 15.5],
                  "beta": [15.5, 30]}

    spectrum = epochs.compute_psd(picks='eeg', fmin=0.5, fmax=30. ,verbose=False)
    psds, freqs = spectrum.get_data(return_freqs=True)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

In [40]:
train_df = []
for epoch in tqdm(train_subset_epoch):
    # 波形をdataframe化
    epoch_df = epoch_to_df(epoch)
    # submit形式のデータフレーム生成
    sub_df = epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=True)

    # パワースペクトル密度計算
    feature_df = pd.DataFrame(eeg_power_band(epoch))

    _df = pd.concat([sub_df, feature_df], axis=1)
    # 必要ないラベルがある場合は除外する
    _df = _df[~_df["condition"].isin(["Sleep stage ?", "Movement time"])]

    train_df.append(_df)

train_df = pd.concat(train_df).reset_index(drop=True)

100%|██████████| 50/50 [02:25<00:00,  2.91s/it]


In [41]:
train_df["condition"].value_counts()

Sleep stage W      34280
Sleep stage 2      21727
Sleep stage R       8207
Sleep stage 1       6912
Sleep stage 3/4     4174
Name: condition, dtype: int64

In [42]:
# ラベルIDに変換
train_df["condition"] = train_df["condition"].map(LABEL2ID)
train_df

,id,meas_time,condition,0,1,2,3,4
0,79cdd2c,1989-04-05 21:48:00,0,0.007536,0.000420,0.000128,0.000070,0.000058
1,79cdd2c,1989-04-05 21:48:30,0,0.007309,0.000552,0.000121,0.000088,0.000081
2,79cdd2c,1989-04-05 21:49:00,0,0.007954,0.000219,0.000049,0.000032,0.000025
3,79cdd2c,1989-04-05 21:49:30,0,0.007967,0.000222,0.000051,0.000038,0.000019
4,79cdd2c,1989-04-05 21:50:00,0,0.007865,0.000282,0.000094,0.000041,0.000021
...,...,...,...,...,...,...,...,...
75295,738f8f4,1989-09-27 09:17:30,0,0.006200,0.000624,0.000289,0.000291,0.000276
75296,738f8f4,1989-09-27 09:18:00,0,0.006041,0.000730,0.000273,0.000276,0.000298
75297,738f8f4,1989-09-27 09:18:30,0,0.005285,0.000812,0.000423,0.000398,0.000419
75298,738f8f4,1989-09-27 09:19:00,0,0.006333,0.000587,0.000312,0.000260,0.000253


In [43]:
test_df = []
for epoch in tqdm(test_whole_epoch):
    # 波形をdataframe化
    epoch_df = epoch_to_df(epoch)
    # submit形式のデータフレーム生成
    sub_df = epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=False)

    # パワースペクトル密度計算
    feature_df = pd.DataFrame(eeg_power_band(epoch))

    _df = pd.concat([sub_df, feature_df], axis=1)

    test_df.append(pd.concat([sub_df, feature_df], axis=1))

test_df = pd.concat(test_df)

100%|██████████| 45/45 [01:35<00:00,  2.11s/it]


## 訓練

In [44]:
# 20％の被験者を選ぶ
val_size = int(train_record_df["subject_id"].nunique() * 0.20)
train_all_subjects = train_record_df["subject_id"].unique()
np.random.shuffle(train_all_subjects)

val_subjects = train_all_subjects[:val_size]
val_ids = train_record_df[train_record_df["subject_id"].isin(val_subjects)]["id"]

In [45]:
# 検証セットを作成します
trn_df = train_df[~train_df["id"].isin(val_ids)]
val_df = train_df[train_df["id"].isin(val_ids)]
trn_df

,id,meas_time,condition,0,1,2,3,4
1604,abce0b7,1989-07-20 20:55:00,0,0.007397,0.000446,0.000159,0.000116,0.000070
1605,abce0b7,1989-07-20 20:55:30,0,0.006923,0.000500,0.000290,0.000176,0.000142
1606,abce0b7,1989-07-20 20:56:00,0,0.006439,0.000663,0.000332,0.000213,0.000212
1607,abce0b7,1989-07-20 20:56:30,0,0.006317,0.000693,0.000358,0.000305,0.000206
1608,abce0b7,1989-07-20 20:57:00,0,0.007086,0.000544,0.000215,0.000145,0.000109
...,...,...,...,...,...,...,...,...
75295,738f8f4,1989-09-27 09:17:30,0,0.006200,0.000624,0.000289,0.000291,0.000276
75296,738f8f4,1989-09-27 09:18:00,0,0.006041,0.000730,0.000273,0.000276,0.000298
75297,738f8f4,1989-09-27 09:18:30,0,0.005285,0.000812,0.000423,0.000398,0.000419
75298,738f8f4,1989-09-27 09:19:00,0,0.006333,0.000587,0.000312,0.000260,0.000253


In [46]:
val_df

,id,meas_time,condition,0,1,2,3,4
0,79cdd2c,1989-04-05 21:48:00,0,0.007536,0.000420,0.000128,0.000070,0.000058
1,79cdd2c,1989-04-05 21:48:30,0,0.007309,0.000552,0.000121,0.000088,0.000081
2,79cdd2c,1989-04-05 21:49:00,0,0.007954,0.000219,0.000049,0.000032,0.000025
3,79cdd2c,1989-04-05 21:49:30,0,0.007967,0.000222,0.000051,0.000038,0.000019
4,79cdd2c,1989-04-05 21:50:00,0,0.007865,0.000282,0.000094,0.000041,0.000021
...,...,...,...,...,...,...,...,...
58675,c879128,1989-07-12 08:48:30,0,0.007581,0.000349,0.000099,0.000067,0.000072
58676,c879128,1989-07-12 08:49:00,0,0.007934,0.000253,0.000053,0.000027,0.000022
58677,c879128,1989-07-12 08:49:30,0,0.007978,0.000176,0.000061,0.000040,0.000026
58678,c879128,1989-07-12 08:50:00,0,0.007857,0.000225,0.000075,0.000033,0.000045


In [47]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(trn_df.iloc[:, 3:], trn_df["condition"])

RandomForestClassifier(random_state=42)

## 結果

In [48]:
val_preds = model.predict(val_df.iloc[:, 3:])

In [49]:
score = accuracy_score(val_df["condition"], val_preds)
print(score)

0.7115614717319773


In [50]:
print(classification_report(val_df["condition"], val_preds,))

              precision    recall  f1-score   support

           0       0.83      0.88      0.86      6506
           1       0.22      0.15      0.18      1109
           2       0.73      0.72      0.73      3906
           3       0.40      0.59      0.48       591
           4       0.44      0.35      0.39      1260

    accuracy                           0.71     13372
   macro avg       0.53      0.54      0.53     13372
weighted avg       0.70      0.71      0.70     13372



## テストの予測

In [51]:
test_df

,id,meas_time,0,1,2,3,4
0,53c1555,1989-11-20 23:19:30,0.007296,0.000352,0.000198,0.000101,0.000120
1,53c1555,1989-11-20 23:20:00,0.006082,0.000668,0.000296,0.000307,0.000291
2,53c1555,1989-11-20 23:20:30,0.006732,0.000522,0.000260,0.000205,0.000187
3,53c1555,1989-11-20 23:21:00,0.007080,0.000389,0.000206,0.000171,0.000148
4,53c1555,1989-11-20 23:21:30,0.007557,0.000386,0.000133,0.000088,0.000056
...,...,...,...,...,...,...,...
906,9b444bb,1989-04-12 07:32:30,0.007375,0.000615,0.000127,0.000087,0.000044
907,9b444bb,1989-04-12 07:33:00,0.007199,0.000614,0.000158,0.000109,0.000080
908,9b444bb,1989-04-12 07:33:30,0.007683,0.000320,0.000105,0.000058,0.000053
909,9b444bb,1989-04-12 07:34:00,0.007481,0.000402,0.000192,0.000072,0.000065


In [52]:
test_preds = model.predict(test_df.iloc[:, 2:])

In [53]:
sample_submission_df["condition"] = test_preds

In [54]:
sample_submission_df["condition"] = sample_submission_df["condition"].map(ID2LABEL)

In [55]:
sample_submission_df

,id,meas_time,condition
0,53c1555,1989-11-20 23:19:30,Sleep stage W
1,53c1555,1989-11-20 23:20:00,Sleep stage W
2,53c1555,1989-11-20 23:20:30,Sleep stage W
3,53c1555,1989-11-20 23:21:00,Sleep stage W
4,53c1555,1989-11-20 23:21:30,Sleep stage W
...,...,...,...
52291,9b444bb,1989-04-12 07:32:30,Sleep stage W
52292,9b444bb,1989-04-12 07:33:00,Sleep stage W
52293,9b444bb,1989-04-12 07:33:30,Sleep stage W
52294,9b444bb,1989-04-12 07:34:00,Sleep stage W


In [56]:
sample_submission_df["condition"].value_counts()

Sleep stage 2      19930
Sleep stage W      19044
Sleep stage R       5894
Sleep stage 1       3977
Sleep stage 3/4     3451
Name: condition, dtype: int64

In [57]:
sample_submission_df.to_csv("submit.csv", index=False)

#### 改善のヒント
* 利用するチャンネルを増やす
  * REM睡眠は眼球運動を伴います。眼の動きを記録したEOGチャンネルを取り入れることで精度向上の可能性があります
* 特徴量エンジニアリングを工夫する
  * 睡眠段階は前後のepochの睡眠段階と強い相関がありそうなので前後の特徴量を考慮できるようにする
  * 特徴量を自動生成してくれるライブラリ等を利用する
* 最先端の手法をためす
    * [Paper with Code](https://paperswithcode.com/sota/sleep-stage-detection-on-sleep-edf)にSleep EDF Expandedのスコアが掲載されています
    * 深層学習を利用したモデルなども提案されています